In [1]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4b13e21514130622928d34214587422f7c2400c30daf0a9b79dc17ceb262d5d6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import bz2
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
import random

In [3]:
# Load the data
# https://www.kaggle.com/code/ghitabenjrinija/sentiment-analysis-on-social-media-posts-with-lstm

training_data = bz2.BZ2File("/content/drive/MyDrive/train.ft.txt.bz2")
training_data = training_data.readlines()
training_data = [x.decode("utf-8") for x in training_data]
print(len(training_data))

3600000


In [4]:
# take the reviews and delete label
training_texts = [re.sub(r"__label__\d ", "", line) for line in training_data]

# choose randomly 100,000 reviews in the original Amazon review dataset
random.seed(42)
random_numbers = random.sample(range(len(training_data)), 100000)
new_dataset = pd.DataFrame({"reviews":training_texts})
new_dataset = new_dataset.iloc[random_numbers]

In [ ]:
# save in csv file
new_dataset.to_csv("Amazon_dataset.csv", index=False)

In [ ]:
file_path = "/content/drive/MyDrive/Amazon_dataset.csv"
new_dataset.to_csv(file_path, index=False)

In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/fake reviews dataset.csv")

In [6]:
RANDOM_SEED = 42

# Set GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
amazon_review = pd.read_csv("/content/drive/MyDrive/Amazon_dataset.csv")

In [8]:
#Load the model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

#Encode reviews in the validation and Amazon set
val_encode = model.encode(df_val['text_'].tolist())
review_encode = model.encode(amazon_review['reviews'].tolist())

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:

#Compute dot score between review embeddings
score_similarity = torch.sum(util.dot_score(review_encode,val_encode),axis = 1).cpu().tolist()

#Combine Amazon reviews & scores
df_ranking = pd.DataFrame({"reviews":amazon_review["reviews"],
                           "Scores": score_similarity})

#Sort by decreasing score
df_ranking = df_ranking.sort_values(by="Scores", ascending=False)



In [ ]:
file_path = '/content/drive/MyDrive/Top_Amazon_dataset.csv'
df_ranking.to_csv(file_path, index=False)

In [10]:
df_ranking.head()

,reviews,Scores
6658,"Story is great, print quality sucks: The story...",1069.730591
40982,Not what I expected: This is one book that did...,1054.442017
1058,Simplistic: I bought this book because a talk ...,1053.736694
58211,Not our favorite: This book was recommended by...,1046.888184
75664,"Not impressed: A lot of misspelled words, and ...",1045.807739
